| [![License: MIT](https://img.shields.io/badge/License-MIT-green.svg)](../LICENSE) | [![Python](https://img.shields.io/badge/Python-3.10+-black.svg)](https://www.python.org/) | [![Jupyter](https://img.shields.io/badge/Jupyter-Notebook-red.svg)](https://jupyter.org/) | [![SQLite](https://img.shields.io/badge/Database-SQLite-darkblue.svg)](https://www.sqlite.org/index.html) | [![Pandas](https://img.shields.io/badge/Data-Pandas-purple.svg)](https://pandas.pydata.org/) | [![NumPy](https://img.shields.io/badge/Math-NumPy-blue.svg)](https://numpy.org/) | [![Matplotlib](https://img.shields.io/badge/Plots-Matplotlib-darkgreen.svg)](https://matplotlib.org/)|
|---|---|---|---|---|---|---|


 | [![Seaborn](https://img.shields.io/badge/Plots-Seaborn-teal.svg)](https://seaborn.pydata.org/) | [![scikit-learn](https://img.shields.io/badge/ML-scikit--learn-orange.svg)](https://scikit-learn.org/stable/) | [![SQLAlchemy](https://img.shields.io/badge/ORM-SQLAlchemy-brown.svg)](https://www.sqlalchemy.org/) |
|---|---|---|


## Notebook 3 – Feature Engineering and Anomaly Detection  

**LuftDataQC: PM2.5 analysis from NILU API – Skøyen and Furulund (2023)**  
Source: [https://api.nilu.no](https://api.nilu.no)  

---

### EN: Project overview  
**Goal:** Engineer time-based features and apply unsupervised anomaly detection (IsolationForest) to identify PM2.5 pollution spikes in 2023 hourly data. Compare results between two contrasting stations in Oslo.  
**Method:** SQLite → pandas → feature creation → scaling → IsolationForest → export.  
**Tools:** Python (`pandas`, `numpy`, `matplotlib`, `seaborn`, `sqlite3`, `SQLAlchemy`, `scikit-learn`)  

### NO: Prosjektoversikt  
**Mål:** Lage tidsbaserte egenskaper og bruke usupervisert avviksdeteksjon (IsolationForest) for å identifisere PM2.5-topper i timesdata for 2023. Sammenligne resultater mellom to kontrasterende stasjoner i Oslo.  
**Metode:** SQLite → pandas → funksjonsutvikling → skalering → IsolationForest → eksport.  
**Verktøy:** Python (`pandas`, `numpy`, `matplotlib`, `seaborn`, `sqlite3`, `SQLAlchemy`, `scikit-learn`)  

---

### Reproducibility - quick reference | Reproduserbarhet - hurtigoversikt  

**Inputs (from Notebook 1):**  
- `data/processed/pm25_2023.sqlite` *(tables: pm25_skøyen, pm25_furulund | tabeller: pm25_skøyen, pm25_furulund)*  

**Outputs (this notebook):**  
- `results/pm25_skøyen_contamination_comparison.png` *(Event comparison - Skøyen | Hendelsessammenligning – Skøyen)*
- `results/pm25_furulund_contamination_comparison.png` *(Event comparison - Furulund | Hendelsessammenligning – Furulund)*
- `results/pm25_skøyen_anomalies.png` *(PM2.5 time series with anomalies - Skøyen| PM2.5-tidsserie med avvik – Skøyen)*
- `results/pm25_furulund_anomalies.png` *(PM2.5 time series with anomalies - Furulund | PM2.5-tidsserie med avvik – Furulund)*  
- `results/pm25_skøyen_with_anomalies.csv` *(Labeled dataset with features and anomaly scores | Datasett med etiketter og avviksscore)*  
- `results/pm25_furulund_with_anomalies.csv` *(Labeled dataset with features and anomaly scores | Datasett med etiketter og avviksscore)*  

**Parameters (this notebook):**  
- **Year** = 2023
- **Pollutant** = PM2.5  
- **Anomaly detection model** = `IsolationForest`, unsupervised  
- **Contamination rate** = 0.01  
- **Rolling windows** = 3h and 12h  

> Time-based features (hour, weekday, weekend) are computed dynamically.

In [ ]:
# (EN) Reproducibility parameters and paths
# (NO) Reproduserbarhetsparametere og stier

YEAR = "2023"
COMPONENT = "PM2.5"

from pathlib import Path

# Define project paths (relative to /notebooks)
PROJECT_ROOT = Path.cwd().parent
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
RESULT_DIR = PROJECT_ROOT / "results"

# Ensure output folders exist
for folder in [RAW_DIR, PROCESSED_DIR, RESULT_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

# Define database path (same as in Notebook 1)
DB_PATH = PROCESSED_DIR / f"pm25_{YEAR}.sqlite"

# Define output paths for this notebook
CONTAMINATION_SKØYEN = RESULT_DIR / "pm25_skøyen_contamination_comparison.png"
CONTAMINATION_FURULUND = RESULT_DIR / "pm25_furulund_contamination_comparison.png"
ANOMALIES_SKØYEN = RESULT_DIR / "pm25_skøyen_anomalies.png"
ANOMALIES_FURULUND = RESULT_DIR / "pm25_furulund_anomalies.png"
EXPORT_SKØYEN = RESULT_DIR / "pm25_skøyen_with_anomalies.csv"
EXPORT_FURULUND = RESULT_DIR / "pm25_furulund_with_anomalies.csv"

In [ ]:
# (EN) Import required libraries and verify data/result folders
# (NO) Importerer nødvendige biblioteker og bekrefter mapper for data/resultater

# Standard libraries
import sqlite3

# Third-party libraries
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Confirm that folders exist
print(f"  • RAW_DIR exists:        {RAW_DIR.exists()}")
print(f"  • PROCESSED_DIR exists:  {PROCESSED_DIR.exists()}")
print(f"  • RESULT_DIR exists:     {RESULT_DIR.exists()}")

In [ ]:
# (EN) Load hourly PM2.5 data from the SQLite database
# (NO) Last inn timebaserte PM2.5-data fra SQLite-databasen

with sqlite3.connect(DB_PATH) as conn:
    skøyen_df   = pd.read_sql('SELECT * FROM "pm25_skøyen"', conn)
    furulund_df = pd.read_sql('SELECT * FROM "pm25_furulund"', conn)

# Show first rows to confirm
display(skøyen_df.head())
display(furulund_df.head())

In [ ]:
# (EN) Convert "fromTime" to timezone-aware datetimes (UTC - Europe/Oslo) and set as index
# (NO) Konverter "fromTime" til tidssone (UTC - Europe/Oslo) og bruk som indeks

# Parse as UTC then convert to Europe/Oslo (preserves tz)

def prepare_station(df, value_name):
    dt = pd.to_datetime(df["fromTime"], utc=True).dt.tz_convert("Europe/Oslo")
    out = pd.DataFrame({value_name: df["value"].astype(float)})
    out.index = dt
    out.index.name = "fromTime"
    return out

skøyen_ts   = prepare_station(skøyen_df,  "pm25_skøyen")
furulund_ts = prepare_station(furulund_df,"pm25_furulund")

# Outer-join on time, ensure monotonic index
df_pm25 = skøyen_ts.join(furulund_ts, how="outer").sort_index()

# Index integrity checks
assert df_pm25.index.tz is not None, "Index must be timezone-aware"
assert df_pm25.index.is_monotonic_increasing, "Index should be time-sorted"
assert not df_pm25.index.duplicated().any(), "Duplicated timestamps found"

display(df_pm25.head(5))

In [ ]:
# (EN) Derive temporal features (hour, weekday, month, weekend) from datetime index
# (NO) Avled tidsegenskaper (time, ukedag, måned, helg) fra datetime-indeksen

idx = df_pm25.index
df_pm25["hour"]       = idx.hour                   # 0..23
df_pm25["weekday"]    = idx.weekday                # 0=Mon..6=Sun
df_pm25["month"]      = idx.month                  # 1..12
df_pm25["is_weekend"] = (df_pm25["weekday"] >= 5).astype(int)

display(df_pm25[["pm25_skøyen","pm25_furulund","hour","weekday","is_weekend"]].head(5))

In [ ]:
# (EN) Compute rolling means (3h, 12h) to smooth short-term variation
# (NO) Beregn rullende gjennomsnitt (3t, 12t) for å glatte korttidsvariasjoner

df_pm25["skøyen_roll_3h"]    = df_pm25["pm25_skøyen"].rolling(3,  min_periods=1).mean()
df_pm25["skøyen_roll_12h"]   = df_pm25["pm25_skøyen"].rolling(12, min_periods=1).mean()
df_pm25["furulund_roll_3h"]  = df_pm25["pm25_furulund"].rolling(3,  min_periods=1).mean()
df_pm25["furulund_roll_12h"] = df_pm25["pm25_furulund"].rolling(12, min_periods=1).mean()

display(df_pm25.filter(regex="skøyen|hour|weekday|is_weekend").head(5))
display(df_pm25.filter(regex="furulund|hour|weekday|is_weekend").head(5))


In [ ]:
# (EN) Run pre-model sanity checks (index integrity, numeric features, valid ranges)
# (NO) Kjør datakontroller før modellering (indeks, numeriske egenskaper, gyldige intervaller)

try:
    # Index checks
    assert df_pm25.index.tz is not None,          "Index must be timezone-aware (Europe/Oslo)"
    assert df_pm25.index.is_monotonic_increasing, "Index must be increasing (sorted)"
    assert not df_pm25.index.duplicated().any(),  "Duplicated timestamps"

    # Feature dtype checks
    needed = [
        "pm25_skøyen","skøyen_roll_3h","skøyen_roll_12h",
        "pm25_furulund","furulund_roll_3h","furulund_roll_12h",
        "hour","weekday","is_weekend","month"
    ]
    non_numeric = [c for c in needed if c in df_pm25 and not pd.api.types.is_numeric_dtype(df_pm25[c])]
    assert not non_numeric, f"Non-numeric columns: {non_numeric}"

    # Valid ranges
    assert set(df_pm25["hour"].dropna().unique())   <= set(range(24)), "hour out of [0..23]"
    assert set(df_pm25["weekday"].dropna().unique()) <= set(range(7)),  "weekday out of [0..6]"
    assert set(df_pm25["is_weekend"].dropna().astype(int).unique()) <= {0,1}, "is_weekend must be 0/1"

    # Non-missing rows per station (for modeling)
    featsskøyenøyen = ["pm25_skøyen","skøyen_roll_3h","skøyen_roll_12h","hour","weekday","is_weekend"]
    feats_furulund   = ["pm25_furulund","furulund_roll_3h","furulund_roll_12h","hour","weekday","is_weekend"]
    assert not df_pm25[featsskøyenøyen].dropna().empty, "No valid Skøyen rows after dropna()"
    assert not df_pm25[feats_furulund].dropna().empty,   "No valid Furulund rows after dropna()"
except AssertionError as e:
    print("Sanity checks: FAILED ->", e)
    raise
else:
    print("Sanity checks: OK")

**Anomaly Detection with IsolationForest | Avviksdeteksjon med IsolationForest**

**(EN)** This section applies an unsupervised machine learning method to detect abnormal spikes in PM2.5 levels. By combining raw values with time-based features, the model identifies observations that deviate from expected patterns.

**(EN)** Dette avsnittet bruker en usupervised maskinlæringsmetode for å oppdage unormale topper i PM2.5-nivåer. Ved å kombinere rådata med tidsbaserte egenskaper identifiserer modellen observasjoner som avviker fra forventede mønstre.

In [ ]:
# (EN) Select minimal, consistent feature sets for each station
# (NO) Velg minimale, konsistente funksjonssett per stasjon

feats_skøyen   = ["pm25_skøyen","skøyen_roll_3h","skøyen_roll_12h","hour","weekday","is_weekend"]
feats_furulund = ["pm25_furulund","furulund_roll_3h","furulund_roll_12h","hour","weekday","is_weekend"]

X_skøyen = df_pm25[feats_skøyen].dropna().copy()
X_furulund   = df_pm25[feats_furulund].dropna().copy()

print(f"Skøyen training rows: {len(X_skøyen):,}")
print(f"Furulund training rows: {len(X_furulund):,}")

In [ ]:
# (EN) Run IsolationForest with multiple contamination values (0.5-5%) to test sensitivity
# (NO) Kjør IsolationForest med flere contamination-verdier (0,5-5 %) for å teste sensitivitet

contamination_values = [0.005, 0.01, 0.02, 0.05]

# Scale features (independently per station)
scaler_skøyen = StandardScaler().fit(X_skøyen)
X_skøyen_scaled = scaler_skøyen.transform(X_skøyen)

scaler_furulund = StandardScaler().fit(X_furulund)
X_furulund_scaled = scaler_furulund.transform(X_furulund)

print("Skøyen — sweep contamination")
for c in contamination_values:
    iso = IsolationForest(contamination=c, random_state=42)
    preds = iso.fit_predict(X_skøyen_scaled)
    rate = (preds == -1).mean()
    print(f"  c={c:.3f} - anomalies: {rate:.2%} (n={len(preds)})")

print("\nFurulund — sweep contamination")
for c in contamination_values:
    iso = IsolationForest(contamination=c, random_state=42)
    preds = iso.fit_predict(X_furulund_scaled)
    rate = (preds == -1).mean()
    print(f"  c={c:.3f} - anomalies: {rate:.2%} (n={len(preds)})")

In [ ]:
# (EN) Visual validation of contamination parameter (Skøyen)
# (NO) Visuell validering av parameteren contamination (Skøyen)

fig, axs = plt.subplots(len(contamination_values), 1, figsize=(15, 12), sharex=True)

for i, contamination in enumerate(contamination_values):
    # Prepare data
    df_subset = df_pm25[[
        "pm25_skøyen", "skøyen_roll_3h", "skøyen_roll_12h",
        "hour", "weekday", "is_weekend"
    ]].dropna().copy()

    # Scale
    X = StandardScaler().fit_transform(df_subset)

    # Fit model
    model = IsolationForest(contamination=contamination, random_state=42)
    preds = model.fit_predict(X)

    # Plot
    axs[i].plot(df_subset.index, df_subset["pm25_skøyen"], color="lightgray", label="Normal data")
    axs[i].scatter(df_subset.index[preds == -1], df_subset["pm25_skøyen"][preds == -1],
                   color="red", s=10, label="Anomaly")

    axs[i].set_title(f"Skøyen - IsolationForest (contamination={contamination})")
    axs[i].set_ylabel("PM2.5 (µg/m³)")
    axs[i].grid(True)
    if i == 0:
        axs[i].legend()

plt.xlabel("Date")
plt.tight_layout()

# Save figure
CONTAMINATION_SKØYEN = RESULT_DIR / "pm25_skøyen_contamination_comparison.png"
plt.savefig(CONTAMINATION_SKØYEN, dpi=300)
plt.show()
print(f"Saved: {CONTAMINATION_SKØYEN}")

####  Visual validation of contamination parameter | Visuell validering av parameteren contamination (Skøyen)

**(EN)** This step compares the IsolationForest model's performance using different contamination values (0.005, 0.01, 0.02, 0.05).  It serves as a **visual validation** to assess the model’s sensitivity and help justify the final parameter choice.

**(NO)** I dette steget sammenlignes IsolationForest-modellen med ulike verdier for contamination (0.005, 0.01, 0.02, 0.05).  Dette fungerer som en **visuell validering** for å vurdere modellens følsomhet og begrunne valg av sluttparameter.

In [ ]:
# (EN) Visual validation of contamination parameter (Furulund)
# (NO) Visuell validering av parameteren contamination (Furulund)

fig, axs = plt.subplots(len(contamination_values), 1, figsize=(15, 12), sharex=True)

for i, contamination in enumerate(contamination_values):
    # Prepare data
    df_subset = df_pm25[[
        "pm25_furulund", "furulund_roll_3h", "furulund_roll_12h",
        "hour", "weekday", "is_weekend"
    ]].dropna().copy()

    # Scale
    X = StandardScaler().fit_transform(df_subset)

    # Fit model
    model = IsolationForest(contamination=contamination, random_state=42)
    preds = model.fit_predict(X)

    # Plot
    axs[i].plot(df_subset.index, df_subset["pm25_furulund"], color="lightgray", label="Normal data")
    axs[i].scatter(df_subset.index[preds == -1], df_subset["pm25_furulund"][preds == -1],
                   color="blue", s=10, label="Anomaly")

    axs[i].set_title(f"Skøyen – IsolationForest (contamination={contamination})")
    axs[i].set_ylabel("PM2.5 (µg/m³)")
    axs[i].grid(True)
    if i == 0:
        axs[i].legend()

plt.xlabel("Date")
plt.tight_layout()

# Save figure
CONTAMINATION_FURULUND = RESULT_DIR / "pm25_furulund_contamination_comparison.png"
plt.savefig(CONTAMINATION_FURULUND, dpi=300)
plt.show()
print(f"Saved: {CONTAMINATION_FURULUND}")

In [ ]:
# (EN) Train final model with contamination=1% and merge anomaly flags back into df_pm25
# (NO) Tren endelig modell med contamination=1 % og flett avviksflagg tilbake i df_pm25

CONTAM = 0.01

iso_skøyen = IsolationForest(contamination=CONTAM, random_state=42).fit(X_skøyen_scaled)
pred_skøyen = iso_skøyen.predict(X_skøyen_scaled)
score_skøyen = iso_skøyen.decision_function(X_skøyen_scaled)

iso_furulund = IsolationForest(contamination=CONTAM, random_state=42).fit(X_furulund_scaled)
pred_furulund = iso_furulund.predict(X_furulund_scaled)
score_furulund = iso_furulund.decision_function(X_furulund_scaled)

# Create labeled frames aligned to original index
lab_skøyen = pd.DataFrame(
    {"anomaly_skøyen": pred_skøyen, "anomaly_score_skøyen": score_skøyen},
    index=X_skøyen.index
)
lab_furulund = pd.DataFrame(
    {"anomaly_furulund": pred_furulund, "anomaly_score_furulund": score_furulund},
    index=X_furulund.index
)

# Merge into df_pm25
df_pm25 = df_pm25.merge(lab_skøyen, left_index=True, right_index=True, how="left")
df_pm25 = df_pm25.merge(lab_furulund, left_index=True, right_index=True, how="left")

# Quick rates on labeled rows
rate_skøyen = (lab_skøyen["anomaly_skøyen"] == -1).mean()
rate_furulund = (lab_furulund["anomaly_furulund"] == -1).mean()
print(f"Final anomaly rate 1% — Skøyen: {rate_skøyen:.2%} | Furulund: {rate_furulund:.2%}")

display(df_pm25.head(8))

In [ ]:
# (EN) Final anomaly plots (1%) - Skøyen and Furulund
# (NO) sluttgraf for avvik (1 %) - Skøyen og Furulund

# Split normal and anomaly points 
df_anomaly = df_pm25[df_pm25["anomaly_skøyen"] == -1]
df_normal  = df_pm25[df_pm25["anomaly_skøyen"] ==  1]
df_furulund_anomaly = df_pm25[df_pm25["anomaly_furulund"] == -1]
df_furulund_normal  = df_pm25[df_pm25["anomaly_furulund"] ==  1]

plt.figure(figsize=(15, 6))

plt.plot(df_pm25.index, df_pm25["pm25_skøyen"],
         color="lightgray", label="Normal data", linewidth=1)

plt.scatter(df_pm25.index[df_pm25["anomaly_skøyen"] == -1],
            df_pm25.loc[df_pm25["anomaly_skøyen"] == -1, "pm25_skøyen"],
            color="red", s=30, label="Anomaly")

plt.title("PM2.5 – Anomaly Detection with IsolationForest\nSkøyen Station, 2023")
plt.xlabel("Date")
plt.ylabel("PM2.5 (µg/m³)")
plt.legend()
plt.grid(True)
plt.tight_layout()

ANOMALIES_SKØYEN = RESULT_DIR / "pm25_skøyen_anomalies.png"
plt.savefig(ANOMALIES_SKØYEN, dpi=300)
plt.show()
print(f"Saved: {ANOMALIES_SKØYEN}")

# Plots - Furulund

plt.figure(figsize=(15, 6))

plt.plot(df_pm25.index, df_pm25["pm25_furulund"],
         color="lightgray", label="Normal data", linewidth=1)

plt.scatter(df_pm25.index[df_pm25["anomaly_furulund"] == -1],
            df_pm25.loc[df_pm25["anomaly_furulund"] == -1, "pm25_furulund"],
            color="blue", s=30, label="Anomaly")

plt.title("PM2.5 – Anomaly Detection with IsolationForest\nFurulund Station, 2023")
plt.xlabel("Date")
plt.ylabel("PM2.5 (µg/m³)")
plt.legend()
plt.grid(True)
plt.tight_layout()

ANOMALIES_FURULUND = RESULT_DIR / "pm25_furulund_anomalies.png"
plt.savefig(ANOMALIES_FURULUND, dpi=300)
plt.show()
print(f"Saved: {ANOMALIES_FURULUND}")


In [ ]:
# (EN) Export labeled CSVs 
# (NO) Eksporter merkede CSV-er 

csv_skøyen = RESULT_DIR / "pm25_skøyen_with_anomalies.csv"
csv_furulund = RESULT_DIR / "pm25_furulund_with_anomalies.csv"
cols_out = [
    "pm25_skøyen","skøyen_roll_3h","skøyen_roll_12h",
    "pm25_furulund","furulund_roll_3h","furulund_roll_12h",
    "hour","weekday","is_weekend","month",
    "anomaly_skøyen","anomaly_score_skøyen",
    "anomaly_furulund","anomaly_score_furulund"
]
df_pm25.to_csv(csv_skøyen, index=True, date_format="%Y-%m-%d %H:%M:%S%z", columns=cols_out)
df_pm25.to_csv(csv_furulund, index=True, date_format="%Y-%m-%d %H:%M:%S%z", columns=cols_out)
print(f"Saved: {csv_skøyen}\nSaved: {csv_furulund}")

#### Conclusion | Konklusjon

**EN: Key findings**
- **Skøyen (urban)** shows more frequent and sharper PM2.5 spikes, especially in winter.
- **Furulund (residential)** shows fewer anomalies and more moderate events.
- Unsupervised **IsolationForest** effectively surfaces candidate anomalies; interpretation requires domain/context (e.g., meteorology, maintenance logs, neighboring stations).

**NO: Hovedfunn**
- **Skøyen (by)** har hyppigere og skarpere PM2.5‑topper, særlig om vinteren.
- **Furulund (boligområde)** har færre og mildere episoder.
- Usupervisert **IsolationForest** fremhever kandidatanomalier; tolkning krever faglig kontekst (meteorologi, vedlikeholdslogger, nabostasjoner).

### Scope and caveats | Omfang og forbehold
- Model is unsupervised; anomaly ≠ error.  
- Results depend on feature set and contamination parameter.  
- Meteorological covariates not modeled here.

_____

### Next Step | Neste steg
Summarize results and visual insights in Notebook 4 (EN) / Notebook 5 (NO).
_____

**Navigation Links**

- [Notebook 1 – Data Collection, Inspection and Storage](notebooks/01_data_sqlite.ipynb)  
- [Notebook 2 – Exploratory Analysis and Quality Checks](notebooks/02_exploratory_qc.ipynb)  
- [Notebook 4 – Summary Report (EN)](notebooks/04_report.ipynb)  
- [Notebook 5 – Sammendragsrapport (NO)](notebooks/05_report_norsk.ipynb)

_____

**References note**
Methodological references (IsolationForest and alternatives) are listed in **Notebook 4 - References**.

In [ ]:
# (EN) Notebook 3 complete
# (NO) Notebook 3 ferdig

print("Notebook 3 complete - outputs saved to results/, ready for Notebook 4 (Summary and Visual Insights).")
print("Notebook 3 ferdig - resultater lagret i results/, klar for Notebook 4 (Oppsummering og visuelle innsikter).")